# <font color = lightblue>Titanic: Machine Learning from Disaster

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

In [2]:
# to-do
# get data by calling the kaggle API
train_path = 'FILE_PATH'
test_path = 'FILE_PATH'

In [3]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
df_train['IsTraining'] = 1
df_test['IsTraining'] = 0

In [4]:
df = pd.concat([df_train, df_test])

In [5]:
df.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsTraining
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S,0
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S,0
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S,0
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C,0
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q,0


In [6]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsTraining
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,1


In [7]:
nameslst = df['Name'].tolist()

In [8]:
title = []
firstname = []
lastname = []
for i in nameslst:
    lastname.append(i.split(',')[0])
    x = i.split(',')[1]
    title.append(x.split('.')[0])
    firstname.append(x.split('.')[1])

In [9]:
names = {'lastname':lastname,
        'title':title,
        'firstname':firstname}

In [10]:
dfnames = pd.DataFrame(names)

In [11]:
df = df.merge(dfnames,left_index = True, right_index = True)

In [12]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsTraining,lastname,title,firstname
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,Braund,Mr,Owen Harris
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,Braund,Mr,Owen Harris
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Cumings,Mrs,John Bradley (Florence Briggs Thayer)
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,Cumings,Mrs,John Bradley (Florence Briggs Thayer)
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Heikkinen,Miss,Laina


### Data Imputation 

In [13]:
by_title_class = df.groupby(['title','Pclass'])

In [14]:
def impute_median(series):
    return series.fillna(series.median())

In [15]:
df.Age = by_title_class.Age.transform(impute_median)

### Convert to numeric

In [16]:
df.Sex = pd.Categorical(df.Sex)
df.title = pd.Categorical(df.title)
df['Gender'] = df.Sex.cat.codes
df['TitleCat'] = df.title.cat.codes

In [17]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,IsTraining,lastname,title,firstname,Gender,TitleCat
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,1,Braund,Mr,Owen Harris,1,11
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,Braund,Mr,Owen Harris,1,11
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,Cumings,Mrs,John Bradley (Florence Briggs Thayer),0,12
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,0,Cumings,Mrs,John Bradley (Florence Briggs Thayer),0,12
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,Heikkinen,Miss,Laina,0,8


### Split df back to training and test

In [18]:
df_test = df[df.IsTraining == 0]
df_train = df[df.IsTraining == 1]

In [19]:
X_train = df_train[['Pclass','Age','Gender','TitleCat']]
y_train = df_train['Survived']
X_test = df_test[['Pclass','Age','Gender','TitleCat']]
y_test = df_test['Survived']

### First Model to Predict Survival

In [20]:
model = LogisticRegression(C = 1e20)
model.fit(X_train,y_train)
mpred = model.predict(X_test)
mproba = model.predict_proba(X_test)
score_1 = model.score(X_test, y_test)
lgloss = log_loss(y_test, mpred)

C:\Users\JosephNicholson\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [21]:
print(f'The mean accuracy for the given test data and labels is {score_1}')
print(f'The log loss for the model is {lgloss}')
print(f'beta is {model.coef_}, and b is {model.intercept_}')

The mean accuracy for the given test data and labels is 0.7957351290684624
The log loss for the model is 7.055133187946634
beta is [[-1.25690164 -0.03665537 -2.56624675 -0.01906564]], and b is [5.16935838]


### Increase Parameters to include Fare

In [26]:
X_train2 = df_train[['Pclass','Age','Gender','TitleCat','Fare']]
X_test2 = df_test[['Pclass','Age','Gender','TitleCat','Fare']]

In [27]:
model_two = LogisticRegression(C = 1e20)
model_two.fit(X_train2,y_train)
mpred_two = model_two.predict(X_test2)
mproba_two = model_two.predict_proba(X_test2)
score_two = model_two.score(X_test2, y_test)
lgloss_two = log_loss(y_test, mpred)

C:\Users\JosephNicholson\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [28]:
print(f'The mean accuracy for the given test data and labels is {score_two}')
print(f'The log loss for the model is {lgloss_two}')
print(f'beta is {model_two.coef_}, and b is {model_two.intercept_}')

The mean accuracy for the given test data and labels is 0.7968574635241302
The log loss for the model is 7.055133187946634
beta is [[-1.24896381e+00 -3.65463388e-02 -2.56426521e+00 -1.92299796e-02
   2.39375556e-04]], and b is [5.14128482]


In [7]:
# to-do 
# Remove Gender as it is similar to title